In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time
import datetime

#HP Tuning
from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard

In [2]:
InputData = np.loadtxt("Data/dataInputLarge.csv",delimiter=",")
OutputData = np.loadtxt ("Data/dataOutputLarge.csv", delimiter=",")

InputDataSmall = np.loadtxt("Data/dataInput2.csv",delimiter=",")
OutputDataSmall = np.loadtxt ("Data/dataOutput2.csv", delimiter=",")

InputDataLarge = np.loadtxt("Data/dataInputLarge2.csv",delimiter=",")
OutputDataLarge = np.loadtxt ("Data/dataOutputLarge2.csv", delimiter=",")

InputDataLarge.shape

(22012, 31)

In [5]:
InputDataTrain = InputData[0:9000]  
InputDataTest = InputData[9000:11006]  
OutputDataTrain = OutputData[0:9000] 
OutputDataTest = OutputData[9000:11006]

InputDataTrainS = InputDataSmall[0:900]  
InputDataTestS = InputDataSmall[900:1106]  
OutputDataTrainS = OutputDataSmall[0:900] 
OutputDataTestS = OutputDataSmall[900:1106]

InputDataTrainL = InputDataLarge[0:18000]  
InputDataTestL = InputDataLarge[18000:22012]  
OutputDataTrainL = OutputDataLarge[0:18000] 
OutputDataTestL = OutputDataLarge[18000:22012]

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))

InputDataTrain_transformed = scaler.fit_transform(InputDataTrain) 
#InputDataTrain_retransformed = scaler.inverse_transform(InputDataTrain_transformed)
InputDataTest_transformed = scaler.fit_transform(InputDataTest)
#InputDataTest_retransformed = scaler.inverse_transform(InputDataTest_transformed)
OutputDataTrain_transformed = scaler.fit_transform(OutputDataTrain)
#OutputDataTrain_retransformed = scaler.inverse_transform(OutputDataTrain_transformed)
OutputDataTest_transformed = scaler.fit_transform(OutputDataTest)
#OutputDataTest_retransformed = scaler.inverse_transform(OutputDataTest_transformed)         


InputDataTrainS_transformed = scaler.fit_transform(InputDataTrainS) 
InputDataTestS_transformed = scaler.fit_transform(InputDataTestS)
OutputDataTrainS_transformed = scaler.fit_transform(OutputDataTrainS)
OutputDataTestS_transformed = scaler.fit_transform(OutputDataTestS)

InputDataTrainL_transformed = scaler.fit_transform(InputDataTrainL) 
InputDataTestL_transformed = scaler.fit_transform(InputDataTestL)
InputDataTestL_retransformed = scaler.inverse_transform(InputDataTestL_transformed)
OutputDataTrainL_transformed = scaler.fit_transform(OutputDataTrainL)
OutputDataTestL_transformed = scaler.fit_transform(OutputDataTestL)
OutputDataTestL_retransformed = scaler.inverse_transform(OutputDataTestL_transformed)     

In [59]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.layers.Dense(62,input_shape = (31,),activation='relu'),
      tf.keras.layers.Dense(31,activation="linear"),
      tf.keras.layers.Dense(31,activation="linear"),
    ])

In [60]:
modelName ="Model"
#modelName ="ModelLarge"

model_load = tf.keras.models.load_model("Models/"+modelName)

In [61]:
model = create_model()
#model = model_load

In [62]:
model.summary()
model.output_shape
#model.layers[1].get_config()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 62)                1984      
_________________________________________________________________
dense_13 (Dense)             (None, 31)                1953      
_________________________________________________________________
dense_14 (Dense)             (None, 31)                992       
Total params: 4,929
Trainable params: 4,929
Non-trainable params: 0
_________________________________________________________________


(None, 31)

In [82]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [83]:
model.fit(x=InputDataTrain_transformed, 
          y=OutputDataTrain_transformed, 
          epochs=2000,
          batch_size = 500,
          validation_data=(InputDataTest_transformed, OutputDataTest_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [84]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 546us/step - loss: 1.2851e-04 - accuracy: 0.8210
Test loss, Test acc: [0.00012851030624005944, 0.8210368752479553]


In [85]:
model.fit(x=InputDataTrainS_transformed, 
          y=OutputDataTrainS_transformed, 
          epochs=1000,
          batch_size = 50,
          validation_data=(InputDataTestS_transformed, OutputDataTestS_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [86]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 533us/step - loss: 3.6685e-04 - accuracy: 0.7906
Test loss, Test acc: [0.0003668488934636116, 0.7906281352043152]


In [87]:
model.fit(x=InputDataTrainL_transformed, 
          y=OutputDataTrainL_transformed, 
          epochs=1000,
          batch_size = 500,
          validation_data=(InputDataTestL_transformed, OutputDataTestL_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [88]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)



63/63 [==============================] - 0s 540us/step - loss: 1.0869e-04 - accuracy: 0.8400
Test loss, Test acc: [0.0001086904012481682, 0.8399800658226013]


In [ ]:
# %tensorboard --logdir logs --host localhost --port 6003

In [23]:
%run Code.ipynb

In [45]:
first = 4010
second = 4011

predictions = model.predict(InputDataTestL_transformed[first:second])
print("predictions shape:", predictions.shape)
print("InputDataTest:",InputDataTestL_transformed[first:second])
print("InputDataTest in db (Command Gains)", InputDataTestL_retransformed[first:second])
print("OutputDataTest:",OutputDataTestL_transformed[first:second])
print("Predictions:",predictions)

print("OutputDataTest in dB:",scaler.inverse_transform(OutputDataTestL_transformed[first:second]))
print("Predictions in dB",scaler.inverse_transform(predictions))

print("Diff:", scaler.inverse_transform(OutputDataTestL_transformed[first:second])-scaler.inverse_transform(predictions))

predictions shape: (1, 31)
InputDataTest: [[1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1.
  1. 0. 0. 0. 1. 1. 1.]]
InputDataTest in db (Command Gains) [[ 12. -12. -12. -12.  12.  12.  12. -12. -12. -12.  12.  12.  12. -12.
  -12. -12.  12.  12.  12. -12. -12. -12.  12.  12.  12. -12. -12. -12.
   12.  12.  12.]]
OutputDataTest: [[0.9869313  0.20718894 0.38389808 0.2095621  0.7919994  0.61294599
  0.79155865 0.20641607 0.38570965 0.20395545 0.79498765 0.61140999
  0.79346564 0.20603459 0.3879075  0.2043642  0.79174284 0.61200254
  0.79551196 0.20753682 0.39186006 0.20873104 0.78956493 0.60907287
  0.79414958 0.21403475 0.37751776 0.1875827  0.76116584 0.63324487
  0.91571561]]
Predictions: [[0.9795226  0.20193663 0.3901816  0.21714607 0.7914475  0.6133987
  0.7894629  0.20715857 0.38685867 0.2101132  0.79404944 0.6060786
  0.7903979  0.20671436 0.39209005 0.20991397 0.7884521  0.6078596
  0.79399234 0.20538858 0.39323336 0.20413297 0.78875446 0.60546803
  0.796

In [46]:
commandGains = InputDataTestL_retransformed[first:second].reshape((31,1)) 
#print(commandGains)
filterGainsPredicted = scaler.inverse_transform(predictions).reshape((31,1)) 
#print(filterGainsPredicted)

In [47]:
thirdOctaveGEQwithPredictions(commandGains,filterGainsPredicted)

C:/Users/Robert/AppData/Local/Temp/xpython_14936/1534437943.py:52: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_14936/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_14936/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wi

In [14]:
model.save("Models/"+modelName)